In this experiment, we adopt BERT for named entity recognition to find methodology, material, research target and metrics in each abstract.

read training dataset, train.json, extract sentences and labels which represents named entities.

In [53]:
import json
file_path = 'train.json'
# Initialize an empty list to store the parsed JSON objects
data_list = []

# Read data from the JSON file line by line
with open(file_path, "r") as file:
    for line in file:
        # Parse each line as a JSON object and add it to the list
        data = json.loads(line)
        data_list.append(data)
'''
binary search for the first element greater than target
'''
def find_first_greater(array, target):
    left, right = 0, len(array) - 1
    result = -1

    while left <= right:
        mid = left + (right - left) // 2
        if array[mid] > target:
            result = mid
            right = mid - 1
        else:
            left = mid + 1

    return result

target_labels = []
target_sentences = []
# for each abstract
for data in data_list:
  target_ner = []
  ner_index = []
  sentences_length = []
  # find the length of each sentence in abstract
  for sentence in data['sentences']:
    sentences_length.append(len(sentence))

  labels = []
  beginning_index = [0]
  for i in range(1,len(sentences_length)):
    beginning_index.append(beginning_index[i-1]+sentences_length[i-1])
    labels.append(['o']*sentences_length[i-1])
  labels.append(['o']*sentences_length[len(sentences_length)-1])

  # construct labels
  j = 0
  for ner_group in data['ner']:
      found = False
      for ner_item in ner_group:
          #if 'method' in ner_item[2].lower() or 'task' in ner_item[2].lower() or 'material' in ner_item[2].lower() or 'generic' in ner_item[2].lower() or 'metric' in ner_item[2].lower() or 'otherscientificterm' in ner_item[2].lower():
          if 'method' in ner_item[2].lower() or 'task' in ner_item[2].lower() or 'material' in ner_item[2].lower() or 'generic' in ner_item[2].lower() or 'metric' in ner_item[2].lower():
              index = find_first_greater(beginning_index, ner_item[0])
              if index == -1:
                index = len(labels)
              if 'method' in ner_item[2].lower():
                for i in range(ner_item[0] - beginning_index[index-1],ner_item[1] - beginning_index[index-1]+1):
                  labels[index-1][i] = 'me'
              elif 'task' in ner_item[2].lower():
                for i in range(ner_item[0] - beginning_index[index-1],ner_item[1] - beginning_index[index-1]+1):
                  labels[index-1][i] = 't'
              elif 'material' in ner_item[2].lower():
                for i in range(ner_item[0] - beginning_index[index-1],ner_item[1] - beginning_index[index-1]+1):
                  labels[index-1][i] = 'ma'
              elif 'generic' in ner_item[2].lower():
                for i in range(ner_item[0] - beginning_index[index-1],ner_item[1] - beginning_index[index-1]+1):
                  labels[index-1][i] = 'ge'
              elif 'metric' in ner_item[2].lower():
                for i in range(ner_item[0] - beginning_index[index-1],ner_item[1] - beginning_index[index-1]+1):
                  labels[index-1][i] = 'met'
              '''elif 'otherscientificterm' in ner_item[2].lower():
                for i in range(ner_item[0] - beginning_index[index-1],ner_item[1] - beginning_index[index-1]+1):
                  labels[index-1][i] = 'ot'
              '''
              found = True
      if found == True:
          target_ner.append(ner_group)
          ner_index.append(j)
      j+=1

  i = 0
  for sentence in data['sentences']:
    if i in ner_index:
      target_sentences.append(sentence)
      target_labels.append(labels[i])
    i+=1


write the sentence and labels into a dataframe

In [54]:
import pandas as pd
target_sentences = [' '.join(sentence) for sentence in target_sentences]
target_labels = [' '.join(label) for label in target_labels]
# Create a DataFrame from the arrays
df = pd.DataFrame({'text': target_sentences, 'labels': target_labels})

# Specify the CSV file name
csv_file = 'output.csv'

# Write the DataFrame to CSV
df.to_csv(csv_file, index=True)

print("Data has been written to", csv_file)

Data has been written to output.csv


In [55]:
%%capture
pip install transformers

In [56]:
from transformers import BertTokenizerFast, BertForTokenClassification
import pandas as pd
import torch
import numpy as np
from torch.optim import SGD
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.utils.class_weight import compute_class_weight

In [57]:
df = pd.read_csv('output.csv')
df.head()

,Unnamed: 0,text,labels
0,0,English is shown to be trans-context-free on t...,ma o o o o o o o o o o o o o o o o o o o o o
1,1,The agreement in question involves number in n...,o ge o o o o o o o o o o o o o o o o o o o o o...
2,2,"The formal proof , which makes crucial use of ...",o o o o o o o o o o me me o o o o o o o o o o ...
3,3,"In this paper , a novel method to learn the in...",o o o o o o me o o o o o o o t t t o o o
4,4,"Based on this assumption , firstly we derived ...",o o o o o o o o o me me me me me me o t t t me...


initialize the tokenizer

In [58]:
tk = BertTokenizerFast.from_pretrained('bert-base-cased')

use $word\_ids$ to identify whether sub-words belong to the same original word

In [59]:
# Flag to determine whether all tokens have been identified
tokens_all_identified = False
# Function to align the labels with the tokenized text
def generate_labels(text_array, label_array):
    # Tokenize the text with a maximum length of 512 and pad as needed
    tokenized_text = tk(text_array, padding='max_length', max_length=512, truncation=True)
    # Get the word IDs from the tokenized text
    text_ids = tokenized_text.word_ids()

    previous = None
    labels = []

    for text_id in text_ids:
        # If the ID is None, assign a padding token ID of -100
        if text_id is None:
            labels.append(-100)
        # if the previous token and current token belongs to different words, set the label to the current word id
        elif text_id != previous:
            try:
                labels.append(label_to_id[label_array[text_id]])
            except:
                labels.append(-100)
        else:
            # if the previous token and current token belongs to the same words, set the label to -100
            try:
                labels.append(label_to_id[label_array[text_id]] if tokens_all_identified else -100)
            except:
                labels.append(-100)
        previous = text_id

    return labels
# process the dataset in batches
class BatchData(torch.utils.data.Dataset):

    def __init__(self, dataframe):

        self.text_array = [tk(str(t), padding='max_length', truncation=True, max_length = 512, return_tensors="pt") for t in dataframe['text'].values.tolist()]
        self.label_array = [generate_labels(x,y) for x,y in zip(dataframe['text'].values.tolist(), [t.split() for t in dataframe['labels'].values.tolist()])]

    def __getitem__(self, index):

        return self.text_array[index], torch.LongTensor(self.label_array[index])
    def __len__(self):

        return len(self.label_array)

construct training and validation dataset

In [60]:
df_subsample = df[0:1000]
labels = []
for l in df_subsample['labels']:
    labels.append(l.split())

# construct unique label set
label_set = set()
for label in labels:
    for l in label:
        if l not in label_set:
            label_set.add(l)

# map the label to id and id to label
label_to_id = {key: value for value, key in enumerate(label_set)}
id_to_label = {value: key for value, key in enumerate(label_set)}

from sklearn.utils import shuffle

train_frac = 0.8
df_shuffled = shuffle(df_subsample, random_state=42)
split_index = int(train_frac * len(df_shuffled))
data_train = df_shuffled.iloc[:split_index]
data_val = df_shuffled.iloc[split_index:]

 note that tokens not representing named entities are more prevalent than those for named entities. To address this data imbalance issue, when calculating the cross-entropy loss, we will set the weight for tokens tokens not belonging to named entities ( 'o' ) to 0.5, and the weight for named entities will be set to 1

In [61]:
from sklearn.utils.class_weight import compute_class_weight
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# Assuming labels are integers starting from 0 to len(unique_labels)-1
# Adjust this according to your labels
class_weights = torch.ones(len(label_set))
# For example, if you wish to make the weight of 'O' label smaller, do something like:
O_label_index = label_to_id["o"]  # Assuming label_to_index is a dictionary that maps label to its index
class_weights[O_label_index] = 0.5  # Halving the weight for 'O' label
class_weights = class_weights.to(device)

define a BERT-based model for token classification

In [62]:
class Bert(torch.nn.Module):

    def __init__(self):

        super(Bert, self).__init__()
        # load the pre-trained BERT model for token classification
        self.bert = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_set))

    def forward(self, input_ids, attention_mask, labels):
        # pass the data and labels to BERT model
        return self.bert(input_ids=input_ids, attention_mask=attention_mask, labels=labels, return_dict=False)

train the model

In [ ]:
def train(m, data_train, data_val):
    # load the data in batches
    train_data_batch = BatchData(data_train)
    val_data_batch = BatchData(data_val)

    train_loader = DataLoader(train_data_batch, batch_size=5, num_workers=4, shuffle=True)
    val_loader = DataLoader(val_data_batch, batch_size=5, num_workers=4)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # apply SGD optimizer
    opt = SGD(m.parameters(), lr=5e-3)

    if torch.cuda.is_available():
        m = m.cuda()

    for i in range(10):

        acc_sum = 0
        loss_sum = 0

        m.train()

        for data, label in tqdm(train_loader):
            # feed label and training dataset to BERT model for classification
            input_ids, attention_mask,labels = data['input_ids'].squeeze(1).to(device), data['attention_mask'].squeeze(1).to(device), label.to(device)

            opt.zero_grad()
            _,results = m(input_ids, attention_mask, labels)
            # calculate cross validation loss
            loss_func = torch.nn.CrossEntropyLoss(weight=class_weights, ignore_index=-100) # Ignoring the -100 label
            l = loss_func(results.view(-1, len(label_set)), labels.view(-1))


            for j in range(results.shape[0]):

              results_clean = results[j][labels[j] != -100]

              pred = results_clean.argmax(dim=1)
              acc_sum += (pred == labels[j][labels[j] != -100]).float().mean()
              loss_sum += l.item()

            l.backward()
            opt.step()

        m.eval()
        train_accuracy = acc_sum / len(data_train)
        average_train_loss = loss_sum / len(data_train)
        
        val_acc_sum = 0
        val_loss_sum = 0
        # calculate the accuracy and loss for validation dataset
        for data, label in val_loader:

            labels = label.to(device)
            l, results = m(data['input_ids'].squeeze(1).to(device), data['attention_mask'].squeeze(1).to(device), labels)

            for j in range(results.shape[0]):

              results_val = results[j][labels[j] != -100]
              val_acc_sum += (results_val.argmax(dim=1) == labels[j][labels[j] != -100]).float().mean()
              val_loss_sum += l.item()

        val_accuracy = val_acc_sum / len(data_val)
        average_val_loss = val_loss_sum / len(data_val)

        print(
            f'epoch: {i + 1} , loss of training dataset: {average_train_loss: .3f}, accuracy of training dataset: {train_accuracy: .3f} , loss of validation dataset: {average_val_loss: .3f}, accuracy of validation dataset: {val_accuracy: .3f}')

m = Bert()
train(m, data_train, data_val)

extract noun chunks, verbs between noun chunks and ADP between noun chunks

In [64]:
import spacy

def extract_verbs_between_noun_chunks(sentence):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(sentence)
    # extract the list of noun chunks
    noun_chunks = list(doc.noun_chunks)
    noun_chunks_except_PRON = []
    for chunk in noun_chunks:
        nouns = [token.i for token in chunk if token.pos_ != 'PRON']
        if nouns:
            # Use indices to create a Span object
            noun_span = doc[nouns[0]: nouns[-1]+1]
            noun_chunks_except_PRON.append(noun_span)
    # extract verbs and ADP between noun chunks
    verbs_between_chunks = []
    ADP_between_chunks = []
    for i in range(len(noun_chunks_except_PRON)-1):
        start = noun_chunks_except_PRON[i].end
        end = noun_chunks_except_PRON[i+1].start

        verbs_between = [token.text for token in doc[start:end] if token.pos_ == 'VERB']
        ADP_between = [token.text for token in doc[start:end] if token.pos_ == 'ADP' or token.pos_ == 'CCONJ']
        verbs_between_chunks.append(verbs_between)
        ADP_between_chunks.append(ADP_between)

    return noun_chunks_except_PRON, verbs_between_chunks,ADP_between_chunks

extract the PROPN chunks from Abstracts

In [65]:
from spacy.tokens import Span
def find_containing_element(str_list, target_string):
    for element in str_list:
        if target_string in element:
            return element
    return None

def find_PROPN(sentence):
  nlp = spacy.load('en_core_web_sm')
  doc = nlp(sentence)
  tokens = word_tokenize(sentence)
  # Initialize an empty Span list
  noun_spans = []

  # Initialize None objects for start and end tokens
  start_token, end_token = None, None
  # Iterate over the tokens
  for token in doc:
      # If this is a 'PROPN' or 'NOUN' token and we haven't seen a 'PROPN' or 'NOUN' before, mark its position
      if (token.pos_ in ['PROPN', 'NOUN']) and start_token is None:
          start_token = token.i
          end_token = token.i
      # If this is a 'PROPN' or 'NOUN' token and we have seen a 'PROPN' or 'NOUN' before, update the end position
      elif (token.pos_ in ['PROPN', 'NOUN']) and start_token is not None:
          end_token = token.i
      # If this is not a 'PROPN' or 'NOUN' token and we have seen a 'PROPN' or 'NOUN' before, create a Span for the sequence
      elif (token.pos_ not in ['PROPN', 'NOUN']) and start_token is not None:
          # Check if any of the tokens in the sequence is a 'PROPN'
          if any(t.pos_ == 'PROPN' for t in doc[start_token:end_token+1]):
              noun_spans.append(Span(doc, start_token, end_token+1))
          start_token, end_token = None, None
  # Check for the last sequence in the Doc
  if start_token is not None and any(t.pos_ == 'PROPN' for t in doc[start_token:end_token+1]):
      noun_spans.append(Span(doc, start_token, end_token+1))

  full_result_array = []
  for span in noun_spans:
      full_result = ""
      for element in span.text.split():
          if element not in tokens:
              result = find_containing_element(tokens, element)
              if result is not None and result not in full_result:
                  full_result+=result+ " "
          else:
              full_result+=element+" "
      full_result_array.append(full_result.strip())
  return full_result_array

predict on testing dataset, return noun chunks and PROPN chunks of corresponding labels

In [66]:
import re
from nltk.tokenize import word_tokenize
'''
The output predictions from BERT include labels for both complete words and subwords generated during tokenization. We need to utilize the $word\_ids$ to identify and discard the subwords and establish a one-to-one relationship between words and their corresponding labels.
'''
def generate_testing_labels(text_array):
    # Tokenize the text with a maximum length of 512 and pad as needed
    tokenized_text = tk(text_array, padding='max_length', max_length=512, truncation=True)
    # Get the word IDs from the tokenized text
    text_ids = tokenized_text.word_ids()

    previous = None
    labels = []

    for text_id in text_ids:
        # If the ID is None, assign a padding token ID of -100
        if text_id is None:
            labels.append(-100)
        # if the previous token and current token belongs to different words, set the label to 1
        elif text_id != previous:
            try:
                labels.append(1)
            except:
                labels.append(-100)
        else:
            # if the previous token and current token belongs to the same words, set the label to -100
            try:
                labels.append(1 if tokens_all_identified else -100)
            except:
                labels.append(-100)
        previous = text_id

    return labels

def evaluate_one_text(m, st):


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if torch.cuda.is_available():
        m = m.cuda()

    text = tk(st, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    labels = torch.Tensor(generate_testing_labels(st)).unsqueeze(0).to(device)

    results = m(text['input_ids'].to(device), text['attention_mask'].to(device), None)
    results = results[0][labels != -100]

    prediction = [id_to_label[i] for i in results.argmax(dim=1).tolist()]
    print(st)
    print(prediction)
    tokens = []
    texts = word_tokenize(st)
    for text in texts:
        tokens += [token for token in re.split(r'([^a-zA-Z0-9])', text) if token]
    # extract the index of corresponding labels
    ma_index = [index for index, value in enumerate(prediction) if value == 'ma']
    t_index = [index for index, value in enumerate(prediction) if value == 't']
    me_index = [index for index, value in enumerate(prediction) if value == 'me']
    ge_index = [index for index, value in enumerate(prediction) if value == 'ge']
    met_index = [index for index, value in enumerate(prediction) if value == 'met']
    other_index = [index for index, value in enumerate(prediction) if value == 'ot']
    # extract tokens between corresponding index
    '''material = [tokens[i] for i in ma_index]
    target = [tokens[i] for i in t_index]
    method = [tokens[i] for i in me_index]
    genric = [tokens[i] for i in ge_index]'''
    metric = [tokens[i] for i in met_index]
    #other = [tokens[i] for i in other_index]
    # extract nouns, verbs and ADP from abstract sentences
    noun_chunks_except_PRON, verbs_between_chunks,ADP_between_chunks = extract_verbs_between_noun_chunks(st)
    # extract PROPN chunks from abstract sentences
    PROPN = find_PROPN(st)
    # extract the material, research target, method from abstract
    matching_material_array = []
    if len(ma_index) > 0:
      for mat in ma_index:
        matching_material = [chunk.text for chunk in noun_chunks_except_PRON if mat>=chunk.start and mat < chunk.end]
        if matching_material:
          if matching_material[0] not in matching_material_array:
            matching_material_array+=matching_material
        else:
          for prop in PROPN:
            if tokens[mat] in prop and prop not in matching_material_array:
              matching_material_array+=[prop]

    matching_target_array = []
    if len(t_index) > 0:
      for ta in t_index:
        matching_target = [chunk.text for chunk in noun_chunks_except_PRON if ta>=chunk.start and ta < chunk.end]
        if matching_target:
          if matching_target[0] not in matching_target_array:
            matching_target_array+=matching_target
        else:
          for prop in PROPN:
            if tokens[ta] in prop and prop not in matching_target_array:
              matching_target_array+=[prop]

    matching_method_array = []
    if len(me_index) > 0:
      for me in me_index:
        matching_method = [chunk.text for chunk in noun_chunks_except_PRON if me>=chunk.start and me < chunk.end]
        if matching_method:
          if matching_method[0] not in matching_method_array:
            matching_method_array+=matching_method
        else:
          for prop in PROPN:
            if tokens[me] in prop and prop not in matching_method_array:
              matching_method_array+=[prop]

    matching_genric_array = []
    if len(ge_index) > 0:
      for ge in ge_index:
        matching_genric = [chunk.text for chunk in noun_chunks_except_PRON if ge>=chunk.start and ge < chunk.end]
        if matching_genric:
          if matching_genric[0] not in matching_genric_array:
            matching_genric_array+=matching_genric
        else:
          for prop in PROPN:
            if tokens[ge] in prop and prop not in matching_genric_array:
              matching_genric_array+=[prop]

    '''matching_metric_array = []
    if len(metric) > 0:
      for me in metric:
        matching_metric = [chunk.text for chunk in noun_chunks_except_PRON if me in chunk.text]
        if matching_metric:
          if matching_metric[0] not in matching_metric_array:
            matching_metric_array+=matching_metric
        else:
          for prop in PROPN:
            if me in prop and prop not in matching_metric_array:
              matching_metric_array+=[prop]'''
    matching_metric_array = metric
    matching_other_array = []
    if len(other_index) > 0:
      for ot in other_index:
        matching_other = [chunk.text for chunk in noun_chunks_except_PRON if ot>=chunk.start and ot < chunk.end]
        if matching_other:
          if matching_other[0] not in matching_other_array:
            matching_other_array+=matching_other
        else:
          for prop in PROPN:
            if tokens[ot] in prop and prop not in matching_other_array:
              matching_other_array+=[prop]
    total_length = len(matching_material_array) + len(matching_target_array) + len(matching_method_array) + len(matching_genric_array) + len(matching_metric_array) + len(matching_other_array)
    #total_length = len(matching_material_array) + len(matching_target_array) + len(matching_method_array) + len(matching_genric_array) + len(matching_metric_array)
    domain_length = 0
    if len(matching_material_array)> 0:
      domain_length +=1
    if len(matching_method_array) > 0:
      domain_length += 1
    if len(matching_genric_array) > 0:
      domain_length += 1
    if len(matching_metric_array) > 0:
      domain_length += 1
    if len(matching_target_array) > 0:
      domain_length += 1
    '''if len(matching_other_array) > 0:
      domain_length += 1'''
    '''print(matching_material_array)
    print(matching_method_array)
    print(matching_genric_array)
    print(matching_metric_array)
    print(matching_target_array)
    print(matching_other_array)'''

    return matching_material_array, matching_target_array, matching_method_array, matching_genric_array,  matching_metric_array,domain_length*2 + total_length

The testing dataset, which contains Abstract from medical, physics and astronomy categories

In [15]:
text2 = "The contraction of the heart muscle is triggered by self-organizing electrical patterns. Abnormalities in these patterns lead to cardiac arrhythmias, a prominent cause of mortality worldwide. The targeted treatment or prevention of arrhythmias requires a thorough understanding of the interacting wavelets, vortices and conduction block sites within the excitation pattern. Currently, there is no conceptual framework that covers the elementary processes during arrhythmogenesis in detail, in particular the transient pivoting patterns observed in patients, which can be interleaved with periods of less fragmented waves. Here, we provide such a framework in terms of quasiparticles and Feynman diagrams, which were originally developed in theoretical physics. We identified three different quasiparticles in excitation patterns: heads, tails and pivots. In simulations and experiments, we show that these basic building blocks can combine into at least four different bound states. By representing their interactions as Feynman diagrams, the creation and annihilation of rotor pairs are shown to be sequences of dynamical creation, annihilation and recombination of the identified quasiparticles. Our results provide a new theoretical foundation for a more detailed theory, analysis and mechanistic insights of topological transitions in excitation patterns, to be applied within and beyond the context of cardiac electrophysiology."

In [16]:
text1 = "Introduction Large healthcare datasets can provide insight that has the potential to improve outcomes for patients. However, it is important to understand the strengths and limitations of such datasets so that the insights they provide are accurate and useful. The aim of this study was to identify data inconsistencies within the Hospital Episodes Statistics (HES) dataset for autistic patients and assess potential biases introduced through these inconsistencies and their impact on patient outcomes. The study can only identify inconsistencies in recording of autism diagnosis and not whether the inclusion or exclusion of the autism diagnosis is the error. Data were extracted from the HES database for the period 1st April 2013 to 31st March 2021 for patients with a diagnosis of autism. First spells in hospital during the study period were identified for each patient and these were linked to any subsequent spell in hospital for the same patient. Data inconsistencies were recorded where autism was not recorded as a diagnosis in a subsequent spell. Features associated with data inconsistencies were identified using a random forest classifiers and regression modelling. Results Data were available for 172,324 unique patients who had been recorded as having an autism diagnosis on first admission. In total, 43.7 % of subsequent spells were found to have inconsistencies. The features most strongly associated with inconsistencies included greater age, greater deprivation, longer time since the first spell, change in provider, shorter length of stay, being female and a change in the main specialty description. The random forest algorithm had an area under the receiver operating characteristic curve of 0.864 in predicting a data inconsistency. For patients who died in hospital, inconsistencies in their final spell were significantly associated with being 80 years and over, being female, greater deprivation and use of a palliative care code in the death spell. conclusions Data inconsistencies in the HES database were relatively common in autistic patients and were associated a number of patient and hospital admission characteristics. Such inconsistencies have the potential to distort our understanding of service use in key demographic groups."

In [17]:
text9 = "Observed scaling relations in galaxies between baryons and dark matter global properties are key to shed light on the process of galaxy formation and on the nature of dark matter. Here, we study the scaling relation between the neutral hydrogen (HI) and dark matter mass in isolated rotationally-supported disk galaxies at low redshift. We first show that state-of-the-art galaxy formation simulations predict that the HI-to-dark halo mass ratio decreases with stellar mass for the most massive disk galaxies. We then infer dark matter halo masses from high-quality rotation curve data for isolated disk galaxies in the local Universe, and report on the actual universality of the HI-to-dark halo mass ratio for these observed galaxies. This scaling relation holds for disks spanning a range of 4 orders of magnitude in stellar mass and 3 orders of magnitude in surface brightness. Accounting for the diversity of rotation curve shapes in our observational fits decreases the scatter of the HI-to-dark halo mass ratio while keeping it constant. This finding extends the previously reported discrepancy for the stellar-to-halo mass relation of massive disk galaxies within galaxy formation simulations to the realm of neutral atomic gas. Our result reveals that isolated galaxies with regularly rotating extended HI disks are surprisingly self-similar up to high masses, which hints at mass-independent self-regulation mechanisms that have yet to be fully understood."

In [18]:
text8 = "The formation history of giant planets inside and outside the solar system remains unknown. We suggest that runaway gas accretion is initiated only at a mass of ~100 M_Earth and that this mass corresponds to the transition to a gas giant, a planet that its composition is dominated in hydrogen and helium. Delaying runaway accretion to later times (a few Myr) and higher masses is likely to be a result of an intermediate stage of efficient heavy-element accretion (at a rate of ~10^-5 M_Earth/yr) that provides sufficient energy to hinder rapid gas accretion. This may imply that Saturn has never reached runaway gas accretion, and that it is a failed giant planet. The transition to a gas giant planet above Saturn's mass naturally explains the differences between the bulk metallicities and internal structures of Jupiter and Saturn. The transition mass to a gas giant planets strongly depends on the exact formation history and birth environment of the planets, which are still not well constrained for our Solar System. In terms of giant exoplanets, delaying runaway gas accretion to planets beyond Saturn's mass can explain the transitions in the mass-radius relations of observed exoplanets and the high metallicity of intermediate-mass exoplanets."

In [19]:
text7 = "Entropy measures quantify the amount of information and correlations present in a quantum system. In practice, when the quantum state is unknown and only copies thereof are available, one must resort to the estimation of such entropy measures. Here we propose a variational quantum algorithm for estimating the von Neumann and Rényi entropies, as well as the measured relative entropy and measured Rényi relative entropy. Our approach first parameterizes a variational formula for the measure of interest by a quantum circuit and a classical neural network, and then optimizes the resulting objective over parameter space. Numerical simulations of our quantum algorithm are provided, using a noiseless quantum simulator. The algorithm provides accurate estimates of the various entropy measures for the examples tested, which renders it as a promising approach for usage in downstream tasks."

In [20]:
text6 = "A challenging aspect of the description of a tokamak disruption is evaluating the hot tail runaway electron (RE) seed that emerges during the thermal quench. This problem is made challenging due to the requirement of describing a strongly non-thermal electron distribution, together with the need to incorporate a diverse range of multiphysics processes including magnetohydrodynamic instabilities, impurity transport, and radiative losses. The present work develops a physics-informed neural network (PINN) tailored to the solution of the hot tail seed during an idealized axisymmetric thermal quench. Here, a PINN is developed to identify solutions to the adjoint relativistic Fokker-Planck equation in the presence of a rapid quench of the plasma's thermal energy. It is shown that the PINN is able to accurately predict the hot tail seed across a range of parameters including the thermal quench time scale, initial plasma temperature, and local current density, in the absence of experimental or simulation data. The hot tail PINN is verified by comparison with a direct Monte Carlo solution, with excellent agreement found across a broad range of thermal quench conditions."

In [21]:
text5 = "The advancement of neutrino observatories has sparked a surge in multi-messenger astronomy. Multiple neutrino associations among blazars are reported while neutrino production sites are located within their central (sub)parsecs. Yet, many questions remain on the nature of those processes. The next generation Event Horizon Telescope (ngEHT) is uniquely positioned for these studies, as its high frequency and resolution can probe both the accretion disk region and the parsec-scale jet. This opens up new opportunities for connecting the two regions and unraveling the proton acceleration and neutrino production in blazars. We outline observational strategies for ngEHT and highlight what it can contribute to the multi-messenger study of blazars."

In [22]:
text4 = "We show that the transformation of a time-evolving spherically symmetric metric tensor into a Painleve-Gullstrand-Lemaitre form brings forth a few curious consequences. The time evolution describes a non-singular gravitational collapse, leading to a bounce and dispersal of all the clustered matter, or a wormhole geometry for certain initial conditions. The null convergence condition is violated only at the onset of bounce or the wormhole formation. As an example, the requirements to develop a Simpson-Visser wormhole/regular black-hole geometry is discussed. The solution can be regarded as a new time-evolving twin of sonic dumb holes found in analog gravity."

In [23]:
text = "Background To gain maximum insight from large administrative healthcare datasets it is important to understand their data quality. Although a gold standard against which to assess criterion validity rarely exists for such datasets, internal consistency can be evaluated. We aimed to identify inconsistencies in the recording of mandatory International Statistical Classification of Diseases and Related Health Problems, tenth revision (ICD- 10) codes within the Hospital Episodes Statistics dataset in England. Methods Three exemplar medical conditions where recording is mandatory once diagnosed were chosen: autism, type II diabetes mellitus and Parkinson’s disease dementia. We identified the first occurrence of the condition ICD-10 code for a patient during the period April 2013 to March 2021 and in subsequent hospital spells. We designed and trained random forest classifiers to identify variables strongly associated with recording inconsistencies. Results For autism, diabetes and Parkinson’s disease dementia respectively, 43.7%, 8.6% and 31.2% of subsequent spells had inconsistencies. Coding inconsistencies were highly correlated with non-coding of an underlying condition, a change in hospital trust and greater time between the spell with the first coded diagnosis and the subsequent spell. For patients with diabetes or Parkinson’s disease dementia, the code recording for spells without an overnight stay were found to have a higher rate of inconsistencies. Conclusions Data inconsistencies are relatively common for the three conditions considered. Where these mandatory diagnoses are not recorded in administrative datasets, and where clinical decisions are made based on such data, there is potential for this to impact patient care."

In [24]:
text3 = "The COVID-19 pandemic has created unprecedented challenges for governments and healthcare systems worldwide, highlighting the critical importance of understanding the factors that contribute to virus transmission. This study aimed to identify the most influential age groups in COVID-19 infection rates at the US county level using the Modified Morris Method and deep learning for time series. Our approach involved training the state-of-the-art time-series model Temporal Fusion Transformer on different age groups as a static feature and the population vaccination status as the dynamic feature. We analyzed the impact of those age groups on COVID-19 infection rates by perturbing individual input features and ranked them based on their Morris sensitivity scores, which quantify their contribution to COVID-19 transmission rates. The findings are verified using ground truth data from the CDC and US Census, which provide the true infection rates for each age group. The results suggest that young adults were the most influential age group in COVID-19 transmission at the county level between March 1, 2020, and November 27, 2021. Using these results can inform public health policies and interventions, such as targeted vaccination strategies, to better control the spread of the virus. Our approach demonstrates the utility of feature sensitivity analysis in identifying critical factors contributing to COVID-19 transmission and can be applied in other public health domains."

In [25]:
test_dataset = []
test_dataset.append(text)
test_dataset.append(text1)
test_dataset.append(text2)
test_dataset.append(text3)
test_dataset.append(text4)
test_dataset.append(text5)
test_dataset.append(text6)
test_dataset.append(text7)
test_dataset.append(text8)
test_dataset.append(text9)

clean the testing dataset

In [26]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
x_clean_test = [re.sub(r'[^a-zA-Z\s]', u'',sentence.replace('\n', ''), flags=re.UNICODE) for sentence in test_dataset]
x_clean_test = [' '.join(list(filter(lambda w: w not in stop_words and len(w)>2, map(lemmatizer.lemmatize, sentence.lower().split())))[:200]) for sentence in x_clean_test]
x_clean_test_array = [list(filter(lambda w: w not in stop_words and len(w)>2, map(lemmatizer.lemmatize, sentence.lower().split())))[:200] for sentence in x_clean_test]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


extract research target, method, material from testing dataset

In [67]:
import nltk
import math
nltk.download('punkt')
sentences = nltk.sent_tokenize(text)
map_sentence_to_keyword_occurance = {}
total_matching_material_array = []
total_matching_target_array = []
total_matching_method_array = []
total_matching_metric_array = []
for sentence in sentences:
  flag = False
  matching_material_array, matching_target_array, matching_method_array, matching_genric_array,  matching_metric_array, total_length = evaluate_one_text(m, sentence)
  '''if len(matching_genric_array) > 0:
    print("this sentence is related to ", matching_genric_array[0])
  else:
    if len(matching_material_array) > 0:
      print("this sentence is related to material use")
      flag = True
    if len(matching_method_array) > 0:
      print("this sentence is related to methodology use")
      flag = True
    if len(matching_metric_array) > 0:
      print("this sentence is related to metric methodology")
      flag = True
    if len(matching_target_array) > 0 and flag == False:
      print("this sentence is related to the topic")
  print("\n")'''
  total_matching_material_array += matching_material_array
  total_matching_target_array += matching_target_array
  total_matching_method_array += matching_method_array
  total_matching_metric_array += matching_metric_array
  map_sentence_to_keyword_occurance[sentence] = total_length
print("material ", total_matching_material_array)
print("research target ",total_matching_target_array)
print("method", total_matching_method_array)
sorted_items = sorted(map_sentence_to_keyword_occurance.items(), key=lambda item: item[1], reverse=True)
sorted_dict = dict(sorted_items)
key_sentence = []
wanted_top_sentences_number = int(len(sorted_dict)/2)+1
if wanted_top_sentences_number>6:
  wanted_top_sentences_number = 6
top_sentences = list(sorted_dict.keys())[:wanted_top_sentences_number]
sentences_order = {}
i = 0
for sentence in sentences:
  sentences_order[sentence] = i
  i+=1
for sentence in top_sentences:
  sentences_order[sentence] = len(sentences_order)
top_sentences = [key for key, value in sentences_order.items() if value == len(sentences_order)]
#print(top_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Background To gain maximum insight from large administrative healthcare datasets it is important to understand their data quality.
['o', 'o', 'o', 'o', 'o', 'o', 'o', 'ma', 'ma', 'ma', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'met', 'o']
Although a gold standard against which to assess criterion validity rarely exists for such datasets, internal consistency can be evaluated.
['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'met', 'met', 'o', 'o', 'o', 'o', 'ge', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
We aimed to identify inconsistencies in the recording of mandatory International Statistical Classification of Diseases and Related Health Problems, tenth revision (ICD- 10) codes within the Hospital Episodes Statistics dataset in England.
['o', 'o', 'o', 'o', 'o', 'o', 'o', 't', 'o', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'ma', 'ma', 'ma', 'ma', 'ma', 'ma', 'ma', 'ma', 'o']
Methods Three exemplar medical conditions where recording is mandatory once di

In [45]:
sentences = nltk.sent_tokenize(text1)
map_sentence_to_keyword_occurance = {}
total_matching_material_array = []
total_matching_target_array = []
total_matching_method_array = []
total_matching_metric_array = []
for sentence in sentences:
  flag = False
  matching_material_array, matching_target_array, matching_method_array, matching_genric_array,  matching_metric_array, total_length = evaluate_one_text(m, sentence)
  total_matching_material_array += matching_material_array
  total_matching_target_array += matching_target_array
  total_matching_method_array += matching_method_array
  total_matching_metric_array += matching_metric_array
  map_sentence_to_keyword_occurance[sentence] = total_length
print("material ", total_matching_material_array)
print("research target ",total_matching_target_array)
print("method", total_matching_method_array)

Introduction Large healthcare datasets can provide insight that has the potential to improve outcomes for patients.
['o', 'ma', 'ma', 'ma', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
However, it is important to understand the strengths and limitations of such datasets so that the insights they provide are accurate and useful.
['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'ge', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
The aim of this study was to identify data inconsistencies within the Hospital Episodes Statistics (HES) dataset for autistic patients and assess potential biases introduced through these inconsistencies and their impact on patient outcomes.
['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'ot', 'o', 'o', 'o', 'ma', 'ma', 'ma', 'ma', 'ma', 'ma', 'ma', 'o', 't', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
The study can only identify inconsistencies in recording of autism diagnosis and not wh

In [46]:
sentences = nltk.sent_tokenize(text2)
map_sentence_to_keyword_occurance = {}
total_matching_material_array = []
total_matching_target_array = []
total_matching_method_array = []
total_matching_metric_array = []
for sentence in sentences:
  matching_material_array, matching_target_array, matching_method_array, matching_genric_array,  matching_metric_array, total_length = evaluate_one_text(m, sentence)
  total_matching_material_array += matching_material_array
  total_matching_target_array += matching_target_array
  total_matching_method_array += matching_method_array
  total_matching_metric_array += matching_metric_array
  map_sentence_to_keyword_occurance[sentence] = total_length
print("material ", total_matching_material_array)
print("research target ",total_matching_target_array)
print("method", total_matching_method_array)

The contraction of the heart muscle is triggered by self-organizing electrical patterns.
['o', 'ot', 'o', 'ot', 'ot', 'ot', 'o', 'o', 'o', 'ot', 'ot', 'ot', 'ot', 'ot', 'o']
Abnormalities in these patterns lead to cardiac arrhythmias, a prominent cause of mortality worldwide.
['o', 'o', 'o', 'ge', 'o', 'o', 't', 't', 'o', 'o', 'o', 't', 'o', 't', 'o', 'o']
The targeted treatment or prevention of arrhythmias requires a thorough understanding of the interacting wavelets, vortices and conduction block sites within the excitation pattern.
['o', 'o', 't', 'o', 't', 't', 't', 'o', 'o', 'o', 'o', 'o', 'o', 'ot', 'ot', 'o', 'ot', 'o', 'ot', 'ot', 'ot', 'o', 'ot', 'ot', 'ot', 'o']
Currently, there is no conceptual framework that covers the elementary processes during arrhythmogenesis in detail, in particular the transient pivoting patterns observed in patients, which can be interleaved with periods of less fragmented waves.
['o', 'o', 'o', 'o', 'o', 'ge', 'ge', 'o', 'o', 'o', 'o', 'ge', 'o', 't

In [47]:
sentences = nltk.sent_tokenize(text3)
map_sentence_to_keyword_occurance = {}
total_matching_material_array = []
total_matching_target_array = []
total_matching_method_array = []
total_matching_metric_array = []
for sentence in sentences:
  matching_material_array, matching_target_array, matching_method_array, matching_genric_array,  matching_metric_array, total_length = evaluate_one_text(m, sentence)
  total_matching_material_array += matching_material_array
  total_matching_target_array += matching_target_array
  total_matching_method_array += matching_method_array
  total_matching_metric_array += matching_metric_array
  map_sentence_to_keyword_occurance[sentence] = total_length
print("material ", total_matching_material_array)
print("research target ",total_matching_target_array)
print("method", total_matching_method_array)

The COVID-19 pandemic has created unprecedented challenges for governments and healthcare systems worldwide, highlighting the critical importance of understanding the factors that contribute to virus transmission.
['o', 't', 't', 't', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 't', 't', 't', 't', 'o']
This study aimed to identify the most influential age groups in COVID-19 infection rates at the US county level using the Modified Morris Method and deep learning for time series.
['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 't', 't', 't', 'o', 't', 'o', 'o', 'o', 'o', 'o', 'o', 'me', 'me', 'me', 'me', 'me', 'me', 'me', 'me', 'o', 'o', 'o']
Our approach involved training the state-of-the-art time-series model Temporal Fusion Transformer on different age groups as a static feature and the population vaccination status as the dynamic feature.
['o', 'ge', 'o', 'o', 'o', 'ge', 'me', 'me', 'me', 'o', 'me', 'o', 'me', 'me'

In [68]:
sentences = nltk.sent_tokenize(text4)
map_sentence_to_keyword_occurance = {}
total_matching_material_array = []
total_matching_target_array = []
total_matching_method_array = []
total_matching_metric_array = []
for sentence in sentences:
  matching_material_array, matching_target_array, matching_method_array, matching_genric_array,  matching_metric_array, total_length = evaluate_one_text(m, sentence)
  total_matching_material_array += matching_material_array
  total_matching_target_array += matching_target_array
  total_matching_method_array += matching_method_array
  total_matching_metric_array += matching_metric_array
  map_sentence_to_keyword_occurance[sentence] = total_length
print("material ", total_matching_material_array)
print("research target ",total_matching_target_array)
print("method", total_matching_method_array)

We show that the transformation of a time-evolving spherically symmetric metric tensor into a Painleve-Gullstrand-Lemaitre form brings forth a few curious consequences.
['o', 'o', 'o', 'o', 'o', 'o', 'o', 'me', 'me', 'o', 'me', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
The time evolution describes a non-singular gravitational collapse, leading to a bounce and dispersal of all the clustered matter, or a wormhole geometry for certain initial conditions.
['o', 'me', 'me', 'o', 'o', 't', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
The null convergence condition is violated only at the onset of bounce or the wormhole formation.
['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
As an example, the requirements to develop a Simpson-Visser wormhole/regular black-hole geometry is discussed.
['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o

In [69]:
sentences = nltk.sent_tokenize(text5)
map_sentence_to_keyword_occurance = {}
total_matching_material_array = []
total_matching_target_array = []
total_matching_method_array = []
total_matching_metric_array = []
for sentence in sentences:
  matching_material_array, matching_target_array, matching_method_array, matching_genric_array,  matching_metric_array, total_length = evaluate_one_text(m, sentence)
  total_matching_material_array += matching_material_array
  total_matching_target_array += matching_target_array
  total_matching_method_array += matching_method_array
  total_matching_metric_array += matching_metric_array
  map_sentence_to_keyword_occurance[sentence] = total_length
print("material ", total_matching_material_array)
print("research target ",total_matching_target_array)
print("method", total_matching_method_array)

The advancement of neutrino observatories has sparked a surge in multi-messenger astronomy.
['o', 'o', 'o', 't', 't', 'o', 'o', 'o', 'o', 'o', 't', 't', 't', 't', 'o']
Multiple neutrino associations among blazars are reported while neutrino production sites are located within their central (sub)parsecs.
['o', 't', 't', 'o', 'o', 'o', 'o', 'o', 'ma', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
Yet, many questions remain on the nature of those processes.
['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'ge', 'o']
The next generation Event Horizon Telescope (ngEHT) is uniquely positioned for these studies, as its high frequency and resolution can probe both the accretion disk region and the parsec-scale jet.
['o', 'o', 'o', 'me', 'me', 'me', 'o', 'me', 'o', 'o', 'o', 'o', 'o', 'o', 'ge', 'o', 'o', 'o', 'o', 'met', 'o', 'met', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
This opens up new opportunities for connecting the two regions and unraveling

In [70]:
sentences = nltk.sent_tokenize(text6)
map_sentence_to_keyword_occurance = {}
total_matching_material_array = []
total_matching_target_array = []
total_matching_method_array = []
total_matching_metric_array = []
for sentence in sentences:
  matching_material_array, matching_target_array, matching_method_array, matching_genric_array,  matching_metric_array, total_length = evaluate_one_text(m, sentence)
  total_matching_material_array += matching_material_array
  total_matching_target_array += matching_target_array
  total_matching_method_array += matching_method_array
  total_matching_metric_array += matching_metric_array
  map_sentence_to_keyword_occurance[sentence] = total_length
print("material ", total_matching_material_array)
print("research target ",total_matching_target_array)
print("method", total_matching_method_array)

A challenging aspect of the description of a tokamak disruption is evaluating the hot tail runaway electron (RE) seed that emerges during the thermal quench.
['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 't', 't', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
This problem is made challenging due to the requirement of describing a strongly non-thermal electron distribution, together with the need to incorporate a diverse range of multiphysics processes including magnetohydrodynamic instabilities, impurity transport, and radiative losses.
['o', 'ge', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'me', 'me', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'me', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
The present work develops a physics-informed neural network (PINN) tailored to the solution of the hot tail seed during an idealized axisymmetric thermal quench.
['o', 'o', 'o', 'o', 'o', 'me', 'me', 'me

In [71]:
sentences = nltk.sent_tokenize(text7)
map_sentence_to_keyword_occurance = {}
total_matching_material_array = []
total_matching_target_array = []
total_matching_method_array = []
total_matching_metric_array = []
for sentence in sentences:
  matching_material_array, matching_target_array, matching_method_array, matching_genric_array,  matching_metric_array, total_length = evaluate_one_text(m, sentence)
  total_matching_material_array += matching_material_array
  total_matching_target_array += matching_target_array
  total_matching_method_array += matching_method_array
  total_matching_metric_array += matching_metric_array
  map_sentence_to_keyword_occurance[sentence] = total_length
print("material ", total_matching_material_array)
print("research target ",total_matching_target_array)
print("method", total_matching_method_array)

Entropy measures quantify the amount of information and correlations present in a quantum system.
['me', 'me', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'me', 'me', 'o']
In practice, when the quantum state is unknown and only copies thereof are available, one must resort to the estimation of such entropy measures.
['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'me', 'me', 'me', 'me', 'me', 'o']
Here we propose a variational quantum algorithm for estimating the von Neumann and Rényi entropies, as well as the measured relative entropy and measured Rényi relative entropy.
['o', 'o', 'o', 'o', 'me', 'me', 'me', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'met', 'met', 'met', 'met', 'met', 'o', 'met', 'met', 'o']
Our approach first parameterizes a variational formula for the measure of interest by a quantum circuit and a classical neural network, and then optimizes the resulting objective over parameter 

In [72]:
sentences = nltk.sent_tokenize(text8)
map_sentence_to_keyword_occurance = {}
total_matching_material_array = []
total_matching_target_array = []
total_matching_method_array = []
total_matching_metric_array = []
for sentence in sentences:
  matching_material_array, matching_target_array, matching_method_array, matching_genric_array,  matching_metric_array, total_length = evaluate_one_text(m, sentence)
  total_matching_material_array += matching_material_array
  total_matching_target_array += matching_target_array
  total_matching_method_array += matching_method_array
  total_matching_metric_array += matching_metric_array
  map_sentence_to_keyword_occurance[sentence] = total_length
print("material ", total_matching_material_array)
print("research target ",total_matching_target_array)
print("method", total_matching_method_array)

The formation history of giant planets inside and outside the solar system remains unknown.
['o', 't', 'o', 'o', 'o', 't', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
We suggest that runaway gas accretion is initiated only at a mass of ~100 M_Earth and that this mass corresponds to the transition to a gas giant, a planet that its composition is dominated in hydrogen and helium.
['o', 'o', 'o', 't', 't', 't', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
Delaying runaway accretion to later times (a few Myr) and higher masses is likely to be a result of an intermediate stage of efficient heavy-element accretion (at a rate of ~10^-5 M_Earth/yr) that provides sufficient energy to hinder rapid gas accretion.
['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'me', 'me', 'o', 'o', 'o', 'me', 'o'

In [73]:
sentences = nltk.sent_tokenize(text9)
map_sentence_to_keyword_occurance = {}
total_matching_material_array = []
total_matching_target_array = []
total_matching_method_array = []
total_matching_metric_array = []
for sentence in sentences:
  matching_material_array, matching_target_array, matching_method_array, matching_genric_array,  matching_metric_array, total_length = evaluate_one_text(m, sentence)
  total_matching_material_array += matching_material_array
  total_matching_target_array += matching_target_array
  total_matching_method_array += matching_method_array
  total_matching_metric_array += matching_metric_array
  map_sentence_to_keyword_occurance[sentence] = total_length
print("material ", total_matching_material_array)
print("research target ",total_matching_target_array)
print("method", total_matching_method_array)

Observed scaling relations in galaxies between baryons and dark matter global properties are key to shed light on the process of galaxy formation and on the nature of dark matter.
['o', 't', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 't', 't', 'o', 'o', 'o', 'o', 'o', 'o', 't', 'o']
Here, we study the scaling relation between the neutral hydrogen (HI) and dark matter mass in isolated rotationally-supported disk galaxies at low redshift.
['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
We first show that state-of-the-art galaxy formation simulations predict that the HI-to-dark halo mass ratio decreases with stellar mass for the most massive disk galaxies.
['o', 'o', 'o', 'o', 'ge', 'me', 'o', 'me', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'met', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
We